In [1]:
import zipfile
import os

# Unzipping the dataset
dataset_zip = "C:/Users/Hp/OneDrive/Desktop/Brain Tumor Detection - v1.0.zip"
extracted_dir = "E:/neuro scan MRI/Brain Tumor Detection Dataset"

with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

In [2]:
import cv2
import numpy as np
import os
from glob import glob

def load_images(image_dir):
    image_files = glob(os.path.join(image_dir, '**/*.jpg'), recursive=True)
    images = []
    labels = []
    
    for file in image_files:
        img = cv2.imread(file)
        img = cv2.resize(img, (128, 128))  # Resize to 128x128 to reduce memory load
        images.append(img)
        
        # Extract label from the file name or directory structure
        if 'no_tumor' in file:
            labels.append(0)
        else:
            labels.append(1)
    
    return np.array(images), np.array(labels)

image_dir = "E:/neuro scan MRI/Brain Tumor Detection Dataset"
X, y = load_images(image_dir)


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np

# Load your dataset
# Assuming X and y are already loaded with your image data and labels

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Normalizing the datasets
X_train = X_train / 255.0
X_test = X_test / 255.0

# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Ensure the data generator is properly configured
train_generator = datagen.flow(X_train, y_train, batch_size=16)

# Load the VGG16 model, excluding the top layers
input_shape = (128, 128, 3)  # Adjust based on your data
vgg = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)

# Adding custom top layers with adjusted complexity
x = Flatten()(vgg.output)
x = Dense(256, activation='relu')(x)  # Reduced the number of neurons
x = Dropout(0.5)(x)  # Increased dropout rate
x = Dense(128, activation='relu')(x)  # Added another layer with fewer neurons
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification output

model = Model(inputs=vgg.input, outputs=output)

# Compile the model with the Adam optimizer
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training the model with early stopping and a reduced number of epochs
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(
    train_generator,
    steps_per_epoch=len(X_train) // 16,
    validation_data=(X_test, y_test),
    epochs=20,  # You can increase this if the model is underfitting
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_accuracy * 100:.2f}%')


Epoch 1/20


c:\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


495/495 ━━━━━━━━━━━━━━━━━━━━ 2336s 5s/step - accuracy: 0.9746 - loss: 0.0477 - val_accuracy: 1.0000 - val_loss: 1.2730e-29
Epoch 2/20


c:\Python312\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


495/495 ━━━━━━━━━━━━━━━━━━━━ 105s 213ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 1.2730e-29
Epoch 3/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 1907s 4s/step - accuracy: 1.0000 - loss: 1.0699e-04 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 109s 221ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 2045s 4s/step - accuracy: 0.9999 - loss: 6.4863e-04 - val_accuracy: 1.0000 - val_loss: 6.1494e-25
Epoch 6/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 118s 239ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 6.1494e-25
Epoch 7/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 3705s 7s/step - accuracy: 1.0000 - loss: 1.6098e-05 - val_accuracy: 1.0000 - val_loss: 7.2709e-14
Epoch 8/20
495/495 ━━━━━━━━━━━━━━━━━━━━ 115s 232ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 7.2709e-14
62/62 ━━━━━━━━━━━━━━━

In [4]:
# Evaluating the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.2f}")

# Generating the confusion matrix
from sklearn.metrics import confusion_matrix, classification_report

y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype("int32")
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# Classification report
print("Classification Report:\n", classification_report(y_test, y_pred))


62/62 ━━━━━━━━━━━━━━━━━━━━ 132s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00
Test Accuracy: 1.00
62/62 ━━━━━━━━━━━━━━━━━━━━ 123s 2s/step
Confusion Matrix:
 [[1980]]
Classification Report:
               precision    recall  f1-score   support

           1       1.00      1.00      1.00      1980

    accuracy                           1.00      1980
   macro avg       1.00      1.00      1.00      1980
weighted avg       1.00      1.00      1.00      1980



c:\Python312\Lib\site-packages\sklearn\metrics\_classification.py:409: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


In [8]:
def predict_image(img_path, model):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (128, 128))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    prediction = model.predict(img)
    return 'Tumor Detected' if prediction > 0.5 else 'No Tumor'

test_image_path = r'E:\neuro scan MRI\Brain Tumor Detection Dataset\Brain Tumor Detection - v1.0\images\volume_369_slice_42_jpg.rf.bad112c9dd09dffdb69a41a59a7acaf1.jpg'
result = predict_image(test_image_path, model)
print(f"Prediction: {result}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 223ms/step
Prediction: Tumor Detected
